In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


In [2]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode="human")


In [3]:
episodes = 5
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        score += reward

        done = terminated or truncated
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:19.0
Episode:2 Score:28.0
Episode:3 Score:24.0
Episode:4 Score:23.0
Episode:5 Score:16.0


In [4]:
env.action_space

Discrete(2)

In [5]:
env.action_space.sample()

1

In [6]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [7]:
env.observation_space.sample()


array([-1.2362918e+00, -1.9311251e+38, -3.9954059e-02,  1.9020681e+38],
      dtype=float32)

In [8]:
log_path = os.path.join('Training', 'Logs')

In [9]:
log_path

'Training\\Logs'

In [10]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [11]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_26
-----------------------------
| time/              |      |
|    fps             | 1396 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 734         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008460904 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00246     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.1         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0183     |
|    value_loss           | 51.4        |
-----------------------------------------
--

In [12]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_CartPole')

In [13]:
model.save(PPO_Path)

In [14]:
PPO_Path

'Training\\Saved Models\\PPO_Model_CartPole'

In [15]:
model.learn(total_timesteps=1000)

Logging to Training\Logs\PPO_27
-----------------------------
| time/              |      |
|    fps             | 1329 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


In [16]:
model = PPO.load(PPO_Path, env=env)

## Evaluation ##

In [17]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

c:\Users\djang\anaconda3\envs\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
c:\Users\djang\anaconda3\envs\Python311\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(500.0, 0.0)

In [18]:
env.close()

## Testing ##

In [19]:
episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs) #now using model
        obs, reward, done, info = env.step(action)
        score += reward

        # done = terminated or truncated
    print('Episode:{} Score:{}'.format(episode, score))
#env.close()

Episode:1 Score:[500.]
Episode:2 Score:[500.]
Episode:3 Score:[299.]
Episode:4 Score:[307.]
Episode:5 Score:[500.]


In [20]:
obs = env.reset()

In [21]:
model.predict(obs)

(array([0], dtype=int64), None)

In [22]:
action, _ = model.predict(obs)

In [23]:
env.action_space.sample()

1

In [24]:
env.step(action)

(array([[-0.04114122, -0.15894823,  0.00884388,  0.2541391 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

## Tensorboard ##

In [25]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [26]:
training_log_path

'Training\\Logs\\PPO_2'

In [27]:
# !tensorboard --logdir={training_log_path}

^C


## Adding a callback to the trianing Stage ##

In [28]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [29]:
save_path = os.path.join('Training', 'Saved Models')

In [31]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=500, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [32]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [33]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_28
-----------------------------
| time/              |      |
|    fps             | 1191 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 660       |
|    iterations           | 2         |
|    time_elapsed         | 6         |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0084747 |
|    clip_fraction        | 0.102     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.686    |
|    explained_variance   | 0.000611  |
|    learning_rate        | 0.0003    |
|    loss                 | 7.1       |
|    n_updates            | 10        |
|    policy_gradient_loss | -0.0149   |
|    value_loss           | 48.8      |
---------------------------------------
--------------------------------------

c:\Users\djang\anaconda3\envs\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=417.80 +/- 67.62
Episode length: 417.80 +/- 67.62
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 418         |
|    mean_reward          | 418         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.006836823 |
|    clip_fraction        | 0.0424      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.612      |
|    explained_variance   | 0.166       |
|    learning_rate        | 0.0003      |
|    loss                 | 25.2        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0115     |
|    value_loss           | 68.6        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 488   |
|    iterations      | 5     |
|    ti

## Changing Policies ##

In [37]:
net_arch = dict(pi=[128,128,128,128], vf=[128,128,128,128])

In [38]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


In [39]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_29
-----------------------------
| time/              |      |
|    fps             | 632  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 370         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015411949 |
|    clip_fraction        | 0.202       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.68       |
|    explained_variance   | -0.00279    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.32        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0232     |
|    value_loss           | 18          |
-----------------------------------------
--

c:\Users\djang\anaconda3\envs\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=436.80 +/- 74.08
Episode length: 436.80 +/- 74.08
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 437        |
|    mean_reward          | 437        |
| time/                   |            |
|    total_timesteps      | 10000      |
| train/                  |            |
|    approx_kl            | 0.01013989 |
|    clip_fraction        | 0.132      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.567     |
|    explained_variance   | 0.547      |
|    learning_rate        | 0.0003     |
|    loss                 | 17         |
|    n_updates            | 40         |
|    policy_gradient_loss | -0.0213    |
|    value_loss           | 47.8       |
----------------------------------------
------------------------------
| time/              |       |
|    fps             | 268   |
|    iterations      | 5     |
|    time_elapsed    | 38    |
|    total_times

## Using an Alternate Algorithm ##

In [40]:
from stable_baselines3 import DQN

In [41]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [42]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4268     |
|    time_elapsed     | 0        |
|    total_timesteps  | 80       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.933    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 973      |
|    time_elapsed     | 0        |
|    total_timesteps  | 142      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.514    |
|    n_updates        | 10       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.857    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 